<a href="https://www.kaggle.com/code/asdkarma/imc-2023-submission?scriptVersionId=138132878" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Submission

A notebook to generate a valid submission. Implements local feature/matcher methods: LoFTR, DISK, and KeyNetAffNetHardNet, Swift, Akaze.


In [1]:
SUBMIT = True
SET = "test" if SUBMIT else "train"
NUM_FEATS = 4096 if SUBMIT else 4096
RESIZE_SMALL_EDGE_TO=800
if SUBMIT:
    DESCRIPTOR = "HARDNET"
else:
    DESCRIPTOR = "HARDNET"
VERBOSE = False
COMPOSITE_DESCRIPTOR = True
UPRIGHT = False

In [2]:
# General utilities
import os
from tqdm import tqdm
from time import time
from fastprogress import progress_bar
import gc
import numpy as np
import pandas as pd
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy

# CV/ML
import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

# 3D reconstruction
import pycolmap


from imc_2023_eval import eval_submission
import pandas as pd
# import openglue

In [3]:
# !cd __notebook_source__.ipynb/kaggle/input/openglue/openglue
# !dir

In [4]:
!git clone https://github.com/ucuapps/OpenGlue.git

Cloning into 'OpenGlue'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 108 (delta 13), reused 10 (delta 10), pack-reused 78
Receiving objects: 100% (108/108), 324.11 KiB | 10.45 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [5]:
import OpenGlue
from OpenGlue import models


In [6]:
print('Kornia version', K.__version__)
print('Pycolmap version', pycolmap.__version__)

LOCAL_FEATURE = 'KeyNetAffNetHardNet'
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device=torch.device('cpu')
# Can be LoFTR, KeyNetAffNetHardNet, or DISK

Kornia version 0.6.11
Pycolmap version 0.3.0


In [7]:
def arr_to_str(a):
    return ';'.join([str(x) for x in a.reshape(-1)])


def load_torch_image(fname, device=torch.device('cpu')):
    img = K.image_to_tensor(cv2.imread(fname), False).float() / 255.
    img = K.color.bgr_to_rgb(img.to(device))
    return img

In [8]:
# We will use ViT global descriptor to get matching shortlists.
def get_global_desc(fnames, model,
                    device =  torch.device('cpu')):
    model = model.eval()
    model= model.to(device)
    config = resolve_data_config({}, model=model)
    transform = create_transform(**config)
    global_descs_convnext=[]
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        img = Image.open(img_fname_full).convert('RGB')
        timg = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            desc = model.forward_features(timg.to(device)).mean(dim=(-1,2))#
            #print (desc.shape)
            desc = desc.view(1, -1)
            desc_norm = F.normalize(desc, dim=1, p=2)
        #print (desc_norm)
        global_descs_convnext.append(desc_norm.detach().cpu())
    global_descs_all = torch.cat(global_descs_convnext, dim=0)
    return global_descs_all


def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs


def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6, # should be strict
                              min_pairs = 20,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)

    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames)

    model = timm.create_model('tf_efficientnet_b7',
                              checkpoint_path='/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b7/1/tf_efficientnet_b7_ra-6c08e654.pth')
    model.eval()
    descs = get_global_desc(fnames, model, device=device)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()
    # removing half
    mask = dm <= sim_th
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]  
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < 1000:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

In [9]:
# Code to manipulate a colmap database.
# Forked from https://github.com/colmap/colmap/blob/dev/scripts/python/database.py

# Copyright (c) 2018, ETH Zurich and UNC Chapel Hill.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#
#     * Neither the name of ETH Zurich and UNC Chapel Hill nor the names of
#       its contributors may be used to endorse or promote products derived
#       from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDERS OR CONTRIBUTORS BE
# LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
# CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
# SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
# INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
# CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.
#
# Author: Johannes L. Schoenberger (jsch-at-demuc-dot-de)

# This script is based on an original implementation by True Price.

import sys
import sqlite3
import numpy as np


IS_PYTHON3 = sys.version_info[0] >= 3

MAX_IMAGE_ID = 2**31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    if IS_PYTHON3:
        return array.tostring()
    else:
        return np.getbuffer(array)


def blob_to_array(blob, dtype, shape=(-1,)):
    if IS_PYTHON3:
        return np.fromstring(blob, dtype=dtype).reshape(*shape)
    else:
        return np.frombuffer(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)


    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = \
            lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = \
            lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = \
            lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = \
            lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = \
            lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = \
            lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(self, model, width, height, params,
                   prior_focal_length=False, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (camera_id, model, width, height, array_to_blob(params),
             prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id,
                  prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2],
             prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),))

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),))

    def add_matches(self, image_id1, image_id2, matches):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),))

    def add_two_view_geometry(self, image_id1, image_id2, matches,
                              F=np.eye(3), E=np.eye(3), H=np.eye(3), config=2):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches), config,
             array_to_blob(F), array_to_blob(E), array_to_blob(H)))

In [10]:
# Code to interface DISK with Colmap.
# Forked from https://github.com/cvlab-epfl/disk/blob/37f1f7e971cea3055bb5ccfc4cf28bfd643fa339/colmap/h5_to_db.py

#  Copyright [2020] [Michał Tyszkiewicz, Pascal Fua, Eduard Trulls]
#
#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.

import os, argparse, h5py, warnings
import numpy as np
from tqdm import tqdm
from PIL import Image, ExifTags


def get_focal(image_path, err_on_default=False):
    image         = Image.open(image_path)
    max_size      = max(image.size)

    exif = image.getexif()
    focal = None
    if exif is not None:
        focal_35mm = None
        # https://github.com/colmap/colmap/blob/d3a29e203ab69e91eda938d6e56e1c7339d62a99/src/util/bitmap.cc#L299
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_35mm = float(value)
                break

        if focal_35mm is not None:
            focal = focal_35mm / 35. * max_size
    
    if focal is None:
        if err_on_default:
            raise RuntimeError("Failed to find focal length")

        # failed to find it in exif, use prior
        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size

    return focal

def create_camera(db, image_path, camera_model):
    image         = Image.open(image_path)
    width, height = image.size

    focal = get_focal(image_path)

    if camera_model == 'simple-pinhole':
        model = 0 # simple pinhole
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1 # pinhole
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2 # simple radial
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == 'opencv':
        model = 4 # opencv
        param_arr = np.array([focal, focal, width / 2, height / 2, 0., 0., 0., 0.])
         
    return db.add_camera(model, width, height, param_arr)


def add_keypoints(db, h5_path, image_path, img_ext, camera_model, single_camera = True):
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')

    camera_id = None
    fname_to_id = {}
    for filename in tqdm(list(keypoint_f.keys())):
        keypoints = keypoint_f[filename][()]

        fname_with_ext = filename# + img_ext
        path = os.path.join(image_path, fname_with_ext)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        if camera_id is None or not single_camera:
            camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(fname_with_ext, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id

def add_matches(db, h5_path, fname_to_id):
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')
    
    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                    continue
            
                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)

                added.add(pair_id)

                pbar.update(1)

In [11]:
# Making kornia local features loading w/o internet
class KeyNetAffNetHardNet(KF.LocalFeature):
    """Convenience module, which implements KeyNet detector + AffNet + HardNet descriptor.

    .. image:: _static/img/keynet_affnet.jpg
    """

    def __init__(
        self,
        num_features: int = 5000,
        upright: bool = False,
        device = torch.device('cpu'),
        scale_laf: float = 1.0,
    ):

        # ORI MODULE
        ori_module = KF.PassLAF() if upright else KF.LAFOrienter(angle_detector=KF.OriNet(False)).eval()
        if not upright:
            weights = torch.load('/kaggle/input/kornia-local-feature-weights/OriNet.pth')['state_dict']
            ori_module.angle_detector.load_state_dict(weights)
        
        # DETECTOR
        detector = KF.KeyNetDetector(
            False, num_features=num_features, ori_module=ori_module, aff_module=KF.LAFAffNetShapeEstimator(False).eval()
        ).to(device)
        kn_weights = torch.load('/kaggle/input/kornia-local-feature-weights/keynet_pytorch.pth',map_location=device)['state_dict']
        detector.model.load_state_dict(kn_weights)
        
        affnet_weights = torch.load('/kaggle/input/kornia-local-feature-weights/AffNet.pth',map_location=device)['state_dict']
        detector.aff.load_state_dict(affnet_weights)
        
        # DESCRIPTOR
        if DESCRIPTOR=="HARDNET":
            patch_descriptor_module = KF.HardNet(False).eval()
            weights = torch.load('/kaggle/input/kornia-local-feature-weights/HardNetLib.pth',map_location=device)['state_dict']  
        elif DESCRIPTOR=="HYNET":
            patch_descriptor_module = KF.HyNet(False).eval()
            weights = torch.load('/kaggle/input/kornia-local-feature-weights/HyNet_LIB.pth',map_location=device)#['state_dict']
        elif DESCRIPTOR=="SOSNET":
            patch_descriptor_module = KF.SOSNet(False).eval()
            weights = torch.load('/kaggle/input/kornia-local-feature-weights/sosnet_32x32_liberty.pth',map_location=device)#['state_dict']
        patch_descriptor_module.load_state_dict(weights)
        descriptor = KF.LAFDescriptor(patch_descriptor_module, patch_size=32, grayscale_descriptor=True).to(device)
        super().__init__(detector, descriptor, scale_laf)


In [12]:
DESCRIPTOR = "HYNET"
HYNET = KeyNetAffNetHardNet(4000, True, device).to(device).eval()
DESCRIPTOR = "HARDNET"

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.


In [13]:
# img_fnames = [f'{src}/{SET}/{x}' for x in data_dict["haiper"]["bike"]]
# # detect_features(img_fnames, 
# #                                 NUM_FEATS,
# #                                 feature_dir= f'featureout/{"haiper"}_{"bike"}',
# #                                 upright=True,
# #                                 device=device,
# #                                 resize_small_edge_to=RESIZE_SMALL_EDGE_TO
# #                                )
# model_ = KF.SIFTFeature()
# x1,x2,y1,y2  = 500,750,250,500
# x1,x2,y1,y2  = 400,1200,200,700

# # x1,x2,y1,y2  = 450,1500,250,1400
# crop = False
# if crop:
#     features = [disk]#[HARDNET1k]
# else:
#     features = [disk]#[HARDNET4k]
    
# for img_path in progress_bar(img_fnames[:1]):
#     img_fname = img_path.split('/')[-1]
#     with torch.inference_mode():
#         timg = load_torch_image(img_path, device=device)
#         if crop:
#             timg=timg[:,:,x1:x2,y1:y2]
#         H, W = timg.shape[2:]
#         print(H,W)
        
#         timg_resized = timg#K.geometry.resize(timg, None, antialias=True)
#         h, w = timg_resized.shape[2:]
#         begin=True
#         for feature in features:
#             _lafs, _resps, _descs = feature(K.color.rgb_to_grayscale(timg_resized))
#             if begin:
#                 lafs,resps,descs = _lafs, _resps, _descs
#                 begin = False
#             else:
#                 lafs, resps,descs = torch.cat((lafs,_lafs),1),torch.cat((resps,_resps),1),torch.cat((descs,_descs),1)
#         lafs[:,:,0,:] *= float(W) / float(w)
#         lafs[:,:,1,:] *= float(H) / float(h)
#         desc_dim = descs.shape[-1]
#         kpts = KF.get_laf_center(lafs).reshape(-1, 2).detach().cpu().numpy()
#         descs = descs.reshape(-1, desc_dim).detach().cpu().numpy()
# print(len(kpts))

In [14]:
# import cv2
# import matplotlib.pyplot as plt

# imageread = cv2.imread(img_path)
  
# # input image is converted to gray scale image
# imagegray = cv2.cvtColor(imageread, cv2.COLOR_BGR2GRAY)
# if crop:
#     imagegray = imagegray[x1:x2,y1:y2]
# pts = cv2.KeyPoint_convert(kpts)

# # drawKeypoints function is used to draw keypoints
# output_image = cv2.drawKeypoints(imagegray, pts, 0, (255, 0, 0),
#                                  flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
  
# # displaying the image with keypoints as the
# # output on the screen
# fig = plt.figure(figsize=(10,10))
# ax = fig.gca()
# ax.set_xticks(np.arange(0, W, 100))
# ax.set_yticks(np.arange(0, H, 100))
# plt.imshow(output_image[x1:x2,y1:y2])
# plt.grid(10)
# # plotting image
# plt.show()

In [15]:
# data = kpts.astype(int)
# data = np.unique(data,axis=0)   
# len(data),len(data)/len(kpts)

In [16]:
# from sklearn.cluster import DBSCAN
# from scipy.spatial.distance import cdist
# clustering = DBSCAN(eps=4, min_samples=4).fit(data)
# np.unique(clustering.labels_)

In [17]:
# dbscan_res = {}
# wcss = {}
# core_indices = clustering.core_sample_indices_
# labels = clustering.labels_
# for idx in range(len(data)):
#     label = labels[idx]
#     if label!=-1:
#         wcss.setdefault(label,{"count":0,"dist":0})
#         wcss[label]["count"]+=1
#         try:
#             wcss[label]["dist"]+=min(cdist(data[core_indices],[data[idx]]))[0]
#         except:
#             continue
# for label in wcss:
#     wcss[label]["avg"]=wcss[label]["dist"]/(wcss[label]["count"]+1)
# wcss

In [18]:
def detect_features(img_fnames,
                    num_feats = 2048,
                    upright = False,
                    device=torch.device('cpu'),
                    feature_dir = '.featureout',
                    resize_small_edge_to = 600):
#     if LOCAL_FEATURE == 'DISK':
        # Load DISK from Kaggle models so it can run when the notebook is offline.
    disk = KF.DISK().to(device)
    pretrained_dict = torch.load('/kaggle/input/disk/pytorch/depth-supervision/1/loftr_outdoor.ckpt', map_location=device)
    disk.load_state_dict(pretrained_dict['extractor'])
    disk.eval()
    HARDNET = KeyNetAffNetHardNet(num_feats, upright, device,).to(device)
    HARDNET.eval()
    SP=torch.load()
    
    if LOCAL_FEATURE == 'KeyNetAffNetHardNet':
        
        if COMPOSITE_DESCRIPTOR:
            features_detectors = {
                "DISK":disk,
                "KAHN":HARDNET,
                "HYNET":HYNET,
    #             "sift":KF.SIFTFeature(num_features=num_feats)
            
            }
        else:
            features_detectors = [KeyNetAffNetHardNet(num_feats, upright, device).to(device).eval()]
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    with h5py.File(f'{feature_dir}/lafs.h5', mode='w') as f_laf, \
         h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='w') as f_desc:
        for img_path in progress_bar(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                timg = load_torch_image(img_path, device=device)
                H, W = timg.shape[2:]
                if resize_small_edge_to is None:
                    timg_resized = timg
                else:
                    timg_resized = K.geometry.resize(timg, resize_small_edge_to, antialias=True)
                h, w = timg_resized.shape[2:]
#                 if LOCAL_FEATURE == 'DISK':
#                     features = disk(timg_resized, num_feats, pad_if_not_divisible=True)[0]
#                     kps1, descs = features.keypoints, features.descriptors
#                     lafs = KF.laf_from_center_scale_ori(kps1[None], torch.ones(1, len(kps1), 1, 1, device=device))
                    
                if LOCAL_FEATURE == 'KeyNetAffNetHardNet':
                    begin = True
                    try:
                        for feature_name,feature in features_detectors.items():
                            if feature_name=="DISK":
                                feats_ = feature(timg_resized, num_feats, pad_if_not_divisible=True)[0]
                                _kps, _descs = feats_.keypoints, torch.unsqueeze(feats_.descriptors,0)
                                _lafs = KF.laf_from_center_scale_ori(_kps[None], torch.ones(1, len(_kps), 1, 1, device=device))
                            else:
                                _lafs, _, _descs = feature(K.color.rgb_to_grayscale(timg_resized))
                            if begin:
                                lafs,descs = _lafs, _descs
                                begin = False
                            else:
                                
                                lafs,descs = torch.cat((lafs,_lafs),1),torch.cat((descs,_descs),1)
                    except Exception as e:
                        print("ERORORORORORORORO",e)
                                    
                lafs[:,:,0,:] *= float(W) / float(w)
                lafs[:,:,1,:] *= float(H) / float(h)
                desc_dim = descs.shape[-1]
                kpts = KF.get_laf_center(lafs).reshape(-1, 2).detach().cpu().numpy()
                descs = descs.reshape(-1, desc_dim).detach().cpu().numpy()
                f_laf[key] = lafs.detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
    return

def get_unique_idxs(A, dim=0):
    # https://stackoverflow.com/questions/72001505/how-to-get-unique-elements-and-their-firstly-appeared-indices-of-a-pytorch-tenso
    unique, idx, counts = torch.unique(A, dim=dim, sorted=True, return_inverse=True, return_counts=True)
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0],device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    return first_indices

def match_features(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=15, 
                   force_mutual = True,
                   matching_alg='smnn'
                  ):
    assert matching_alg in ['smnn', 'adalam']
    with h5py.File(f'{feature_dir}/lafs.h5', mode='r') as f_laf, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='r') as f_desc, \
        h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:

        for pair_idx in progress_bar(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            lafs1 = torch.from_numpy(f_laf[key1][...]).to(device)
            lafs2 = torch.from_numpy(f_laf[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
            if matching_alg == 'adalam':
                img1, img2 = cv2.imread(fname1), cv2.imread(fname2)
                hw1, hw2 = img1.shape[:2], img2.shape[:2]
                adalam_config = KF.adalam.get_adalam_default_config()
                #adalam_config['orientation_difference_threshold'] = None
                #adalam_config['scale_rate_threshold'] = None
                adalam_config['force_seed_mnn']= False
                adalam_config['search_expansion'] = 16
                adalam_config['ransac_iters'] = 128
                adalam_config['device'] = device
                dists, idxs = KF.match_adalam(desc1, desc2,
                                              lafs1, lafs2, # Adalam takes into account also geometric information
                                              hw1=hw1, hw2=hw2,
                                              config=adalam_config) # Adalam also benefits from knowing image size
            else:
                dists, idxs = KF.match_smnn(desc1, desc2, 0.98)
            if len(idxs)  == 0:
                continue
            # Force mutual nearest neighbors
            if force_mutual:
                first_indices = get_unique_idxs(idxs[:,1])
                idxs = idxs[first_indices]
                dists = dists[first_indices]
            n_matches = len(idxs)
            if False:
                print (f'{key1}-{key2}: {n_matches} matches')
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
    return

def match_loftr(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout_loftr',
                   device=torch.device('cpu'),
                   min_matches=15, resize_to_ = (640, 480)):
    matcher = KF.LoFTR(pretrained=None)
    matcher.load_state_dict(torch.load('/kaggle/input/loftr/pytorch/outdoor/1/loftr_outdoor.ckpt')['state_dict'])
    matcher = matcher.to(device).eval()

    # First we do pairwise matching, and then extract "keypoints" from loftr matches.
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='w') as f_match:
        for pair_idx in progress_bar(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            # Load img1
            timg1 = K.color.rgb_to_grayscale(load_torch_image(fname1, device=device))
            H1, W1 = timg1.shape[2:]
            if H1 < W1:
                resize_to = resize_to_[1], resize_to_[0]
            else:
                resize_to = resize_to_
            timg_resized1 = K.geometry.resize(timg1, resize_to, antialias=True)
            h1, w1 = timg_resized1.shape[2:]

            # Load img2
            timg2 = K.color.rgb_to_grayscale(load_torch_image(fname2, device=device))
            H2, W2 = timg2.shape[2:]
            if H2 < W2:
                resize_to2 = resize_to[1], resize_to[0]
            else:
                resize_to2 = resize_to_
            timg_resized2 = K.geometry.resize(timg2, resize_to2, antialias=True)
            h2, w2 = timg_resized2.shape[2:]
            with torch.inference_mode():
                input_dict = {"image0": timg_resized1,"image1": timg_resized2}
                correspondences = matcher(input_dict)
            mkpts0 = correspondences['keypoints0'].cpu().numpy()
            mkpts1 = correspondences['keypoints1'].cpu().numpy()

            mkpts0[:,0] *= float(W1) / float(w1)
            mkpts0[:,1] *= float(H1) / float(h1)

            mkpts1[:,0] *= float(W2) / float(w2)
            mkpts1[:,1] *= float(H2) / float(h2)

            n_matches = len(mkpts1)
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=np.concatenate([mkpts0, mkpts1], axis=1))

    # Let's find unique loftr pixels and group them together.
    kpts = defaultdict(list)
    match_indexes = defaultdict(dict)
    total_kpts=defaultdict(int)
    with h5py.File(f'{feature_dir}/matches_loftr.h5', mode='r') as f_match:
        for k1 in f_match.keys():
            group  = f_match[k1]
            for k2 in group.keys():
                matches = group[k2][...]
                total_kpts[k1]
                kpts[k1].append(matches[:, :2])
                kpts[k2].append(matches[:, 2:])
                current_match = torch.arange(len(matches)).reshape(-1, 1).repeat(1, 2)
                current_match[:, 0]+=total_kpts[k1]
                current_match[:, 1]+=total_kpts[k2]
                total_kpts[k1]+=len(matches)
                total_kpts[k2]+=len(matches)
                match_indexes[k1][k2]=current_match

    for k in kpts.keys():
        kpts[k] = np.round(np.concatenate(kpts[k], axis=0))
    unique_kpts = {}
    unique_match_idxs = {}
    out_match = defaultdict(dict)
    for k in kpts.keys():
        uniq_kps, uniq_reverse_idxs = torch.unique(torch.from_numpy(kpts[k]),dim=0, return_inverse=True)
        unique_match_idxs[k] = uniq_reverse_idxs
        unique_kpts[k] = uniq_kps.numpy()
    for k1, group in match_indexes.items():
        for k2, m in group.items():
            m2 = deepcopy(m)
            m2[:,0] = unique_match_idxs[k1][m2[:,0]]
            m2[:,1] = unique_match_idxs[k2][m2[:,1]]
            mkpts = np.concatenate([unique_kpts[k1][ m2[:,0]],
                                    unique_kpts[k2][  m2[:,1]],
                                   ],
                                   axis=1)
            unique_idxs_current = get_unique_idxs(torch.from_numpy(mkpts), dim=0)
            m2_semiclean = m2[unique_idxs_current]
            unique_idxs_current1 = get_unique_idxs(m2_semiclean[:, 0], dim=0)
            m2_semiclean = m2_semiclean[unique_idxs_current1]
            unique_idxs_current2 = get_unique_idxs(m2_semiclean[:, 1], dim=0)
            m2_semiclean2 = m2_semiclean[unique_idxs_current2]
            out_match[k1][k2] = m2_semiclean2.numpy()
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp:
        for k, kpts1 in unique_kpts.items():
            f_kp[k] = kpts1
    
    with h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for k1, gr in out_match.items():
            group  = f_match.require_group(k1)
            for k2, match in gr.items():
                group[k2] = match
    return

def import_into_colmap(img_dir,
                       feature_dir ='.featureout',
                       database_path = 'colmap.db',
                       img_ext='.jpg'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, img_ext, 'simple-radial', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )

    db.commit()
    return

In [19]:
src = '/kaggle/input/image-matching-challenge-2023'

In [20]:
# Get data from csv.

data_dict = {}
if SUBMIT:
    sample_submission_file = f'{src}/sample_submission.csv'
else:
    sample_submission_file = f'{src}/train/train_labels.csv'
    
sample_df = pd.read_csv(sample_submission_file)


for i,j in sample_df.iterrows():
    dataset = j["dataset"]
    scene = j["scene"]
    
    data_dict.setdefault(dataset,{}).setdefault(scene,[]).append(j["image_path"])

if False:
    for dataset,value in data_dict.items():
        for scene,image_dir in value.items():
            scene_dir = f"{src}/{SET}/{dataset}/{scene}"
            if "images_full" in os.listdir(scene_dir):

                image_dir = f"{scene_dir}/images_full"
                for image in os.listdir(image_dir):
                    data_dict[dataset][scene].append(os.path.join(image_dir,image))

if not SUBMIT:
    a = data_dict["heritage"].pop("dioscuri")
#     b = data_dict.pop("haiper")
for dataset in data_dict:
    for scene in data_dict[dataset]:
        print(f'{dataset} / {scene} -> {len(data_dict[dataset][scene])} images')

2cfa01ab573141e4 / 2fa124afd1f74f38 -> 3 images


In [21]:
def main_func():
    gc.collect()
    datasets = []
    for dataset in data_dict:
        datasets.append(dataset)

    for dataset in datasets:
        print(dataset)
        if dataset not in out_results:
            out_results[dataset] = {}
        for scene in data_dict[dataset]:
            print(scene)
            # Fail gently if the notebook has not been submitted and the test data is not populated.
            # You may want to run this on the training data in that case?
            img_dir = f'{src}/{SET}/{dataset}/{scene}/images'
            if not os.path.exists(img_dir):
                continue
            # Wrap the meaty part in a try-except block.
            try:
                out_results[dataset][scene] = {}
                img_fnames = [f'{src}/{SET}/{x}' for x in data_dict[dataset][scene]]
                print (f"Got {len(img_fnames)} images")
                feature_dir = f'featureout/{dataset}_{scene}'
                if not os.path.isdir(feature_dir):
                    os.makedirs(feature_dir, exist_ok=True)
                t=time()
                index_pairs = get_image_pairs_shortlist(img_fnames,
                                      sim_th = 0.5, # should be strict
                                      min_pairs = 20, # we select at least min_pairs PER IMAGE with biggest similarity
                                      exhaustive_if_less = 20,
                                      device=device)
                
                t=time() -t 
                timings['shortlisting'].append(t)
                print (f'{len(index_pairs)}, pairs to match, {t:.4f} sec')
                gc.collect()
                t=time()
                if LOCAL_FEATURE != 'LoFTR':
                    detect_features(img_fnames, 
                                NUM_FEATS,
                                feature_dir=feature_dir,
                                upright=UPRIGHT,
                                device=device,
                                resize_small_edge_to=RESIZE_SMALL_EDGE_TO
                               )
                    gc.collect()
                    t=time() -t 
                    timings['feature_detection'].append(t)
                    print(f'Features detected in  {t:.4f} sec')
                    t=time()
                    match_features(img_fnames, index_pairs, feature_dir=feature_dir,device=device,matching_alg='smnn')
                else:
                    match_loftr(img_fnames, index_pairs, feature_dir=feature_dir, device=device, resize_to_=(600, 800))
                t=time() -t 
                timings['feature_matching'].append(t)
                print(f'Features matched in  {t:.4f} sec')
                database_path = f'{feature_dir}/colmap.db'
                if os.path.isfile(database_path):
                    os.remove(database_path)
                gc.collect()
                import_into_colmap(img_dir, feature_dir=feature_dir,database_path=database_path)
                output_path = f'{feature_dir}/colmap_rec_{LOCAL_FEATURE}'

                t=time()
                pycolmap.match_exhaustive(database_path,verbose=VERBOSE)
                t=time() - t 
                timings['RANSAC'].append(t)
                print(f'RANSAC in  {t:.4f} sec')

                t=time()
                # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
                mapper_options = pycolmap.IncrementalMapperOptions()
                mapper_options.min_model_size = 3
                os.makedirs(output_path, exist_ok=True)
                maps = pycolmap.incremental_mapping(database_path=database_path, image_path=img_dir, output_path=output_path, options=mapper_options,)
                print(maps)
                #clear_output(wait=False)
                t=time() - t
                timings['Reconstruction'].append(t)
                print(f'Reconstruction done in  {t:.4f} sec')
                imgs_registered  = 0
                best_idx = None
#                 print ("Looking for the best reconstruction")
                if isinstance(maps, dict):
                    for idx1, rec in maps.items():
#                         print (idx1, rec.summary())
                        if len(rec.images) > imgs_registered:
                            imgs_registered = len(rec.images)
                            best_idx = idx1
                if best_idx is not None:
#                     print (maps[best_idx].summary())
                    for k, im in maps[best_idx].images.items():
                        key1 = f'{dataset}/{scene}/images/{im.name}'
                        out_results[dataset][scene][key1] = {}
                        out_results[dataset][scene][key1]["R"] = deepcopy(im.rotmat())
                        out_results[dataset][scene][key1]["t"] = deepcopy(np.array(im.tvec))
                print(f'Registered: {dataset} / {scene} -> {len(out_results[dataset][scene])} images')
                print(f'Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images')
                create_submission(out_results, data_dict)
                gc.collect()
            except:
                pass

In [22]:
def assert_func(s:str)->bool:
    s = s.lower()
    if "n" in s or ";;" in s:
        return False
    return True

In [23]:
# Function to create a submission file.
def create_submission(data_dict):
    
    main_func()
    with open(f'submission.csv', 'w') as f:
        f.write('image_path,dataset,scene,rotation_matrix,translation_vector\n')
        for dataset in data_dict:
            if dataset in out_results:
                res = out_results[dataset]
            else:
                res = {}
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}
                for image in data_dict[dataset][scene]:
                    if "full" not in image:
                        try:
                            if image in scene_res:
                                print (image)
                                R = scene_res[image]['R'].reshape(-1)
                                T = scene_res[image]['t'].reshape(-1)
                            else:
                                R = np.eye(3).reshape(-1)
                                T = np.zeros((3))

                            s1 = arr_to_str(R)
                            s2 = arr_to_str(T)
                            assert assert_func(s1) and assert_func(s2)

                        except:
                            R = np.eye(3).reshape(-1)
                            T = np.zeros((3))
                        f.write(f'{image},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n')

In [24]:
out_results = {}
timings = {"shortlisting":[],
               "feature_detection": [],
               "feature_matching":[],
               "RANSAC": [],
               "Reconstruction": []}
if SUBMIT:
    create_submission(data_dict)

2cfa01ab573141e4
2fa124afd1f74f38


In [25]:

# if not SUBMIT:        
#     rotation_thresholds_degrees_dict = {
#         **{('haiper', scene): np.linspace(0.2, 10, 10) for scene in ['bike', 'chairs', 'fountain']},
#         **{('heritage', scene): np.linspace(0.2, 10, 10) for scene in ['cyprus', 'dioscuri']},
#         **{('heritage', 'wall'): np.linspace(0.2, 10, 10)},
#         **{('urban', 'kyiv-puppet-theater'): np.linspace(0.2, 10, 10)},
#     }

#     translation_thresholds_meters_dict = {
#         **{('haiper', scene): np.geomspace(0.05, 1, 10) for scene in ['bike', 'chairs', 'fountain']},
#         **{('heritage', scene): np.geomspace(0.05, 1, 10) for scene in ['cyprus', 'dioscuri']},
#         **{('heritage', 'wall'): np.geomspace(0.05, 1, 10)},
#         **{('urban', 'kyiv-puppet-theater'): np.geomspace(0.5, 1, 10)},
#     }
    
#     df = pd.read_csv("/kaggle/input/image-matching-challenge-2023/train/train_labels.csv")
#     df = df[['image_path','dataset', 'scene',  'rotation_matrix',
#            'translation_vector']]
#     df = df[df["scene"]!="dioscuri"]
#     df.to_csv("train.csv",index=False)
    
#     eval_submission(submission_csv_path='submission.csv',
#                ground_truth_csv_path='train.csv',
#                rotation_thresholds_degrees_dict=rotation_thresholds_degrees_dict,
#                translation_thresholds_meters_dict=translation_thresholds_meters_dict,
#                verbose=True)

In [26]:
if not SUBMIT:
# Get data from csv.
    data_dict = {}
    if SUBMIT:
        sample_submission_file = f'{src}/sample_submission.csv'
    else:
        sample_submission_file = f'{src}/train/train_labels.csv'

    sample_df = pd.read_csv(sample_submission_file)


    for i,j in sample_df.iterrows():
        dataset = j["dataset"]
        scene = j["scene"]

        data_dict.setdefault(dataset,{}).setdefault(scene,[]).append(j["image_path"])

    if False:
        for dataset,value in data_dict.items():
            for scene,image_dir in value.items():
                scene_dir = f"{src}/{SET}/{dataset}/{scene}"
                if "images_full" in os.listdir(scene_dir):

                    image_dir = f"{scene_dir}/images_full"
                    for image in os.listdir(image_dir):
                        data_dict[dataset][scene].append(os.path.join(image_dir,image))

    if not SUBMIT:
        a = data_dict["heritage"].pop("dioscuri")
#         b = data_dict.pop("haiper")
#         b = data_dict.pop("heritage")
    for dataset in data_dict:
        for scene in data_dict[dataset]:
            print(f'{dataset} / {scene} -> {len(data_dict[dataset][scene])} images')
    
        
    rotation_thresholds_degrees_dict = {
        **{('haiper', scene): np.linspace(0.2, 10, 10) for scene in ['bike', 'chairs', 'fountain']},
        **{('heritage', scene): np.linspace(0.2, 10, 10) for scene in ['cyprus', 'dioscuri']},
        **{('heritage', 'wall'): np.linspace(0.2, 10, 10)},
        **{('urban', 'kyiv-puppet-theater'): np.linspace(0.2, 10, 10)},
    }

    translation_thresholds_meters_dict = {
        **{('haiper', scene): np.geomspace(0.05, 1, 10) for scene in ['bike', 'chairs', 'fountain']},
        **{('heritage', scene): np.geomspace(0.05, 1, 10) for scene in ['cyprus', 'dioscuri']},
        **{('heritage', 'wall'): np.geomspace(0.05, 1, 10)},
        **{('urban', 'kyiv-puppet-theater'): np.geomspace(0.5, 1, 10)},
    }
    
    df = pd.read_csv("/kaggle/input/image-matching-challenge-2023/train/train_labels.csv")
    df = df[['image_path','dataset', 'scene',  'rotation_matrix',
           'translation_vector']]
    df = df[df["scene"]!="dioscuri"]
#     df = df[df["dataset"]!="heritage"]
#     df = df[df["dataset"]!="haiper"]
    df.to_csv("train.csv",index=False)
    
    

In [27]:
if not SUBMIT:
    expr = {}
    import itertools
    descriptors = ["HARDNET"]
    feats = [2096]
    sizes = [800]
    composite_status = [True]
    upright_ = [False]
    
    for DESCRIPTOR,NUM_FEATS,RESIZE_SMALL_EDGE_TO,COMPOSITE_DESCRIPTOR,UPRIGHT in itertools.product(*[descriptors,feats,sizes,composite_status,upright_]):
        out_results = {}
        timings = {"shortlisting":[],
                       "feature_detection": [],
                       "feature_matching":[],
                       "RANSAC": [],
                       "Reconstruction": []}
        t = time()
        create_submission(data_dict)
        t = time()-t
        expr[(DESCRIPTOR,NUM_FEATS,RESIZE_SMALL_EDGE_TO,f"COMPOSITE_DESCRIPTOR_{str(COMPOSITE_DESCRIPTOR)}",f"UPRIGHT_{str(UPRIGHT)}")] = (eval_submission(submission_csv_path='submission.csv',
               ground_truth_csv_path='train.csv',
               rotation_thresholds_degrees_dict=rotation_thresholds_degrees_dict,
               translation_thresholds_meters_dict=translation_thresholds_meters_dict,
               verbose=True),t)
        
    print(expr)

In [28]:
# {('HARDNET', 4096, 800, 'COMPOSITE_DESCRIPTOR_False', 'UPRIGHT_True'): (0.21076925, 149.39732480049133), ('HARDNET', 4096, 800, 'COMPOSITE_DESCRIPTOR_False', 'UPRIGHT_False'): (0.32892305, 146.10149264335632)}